<a href="https://colab.research.google.com/github/ShashankShorya0211/MIMDPU/blob/main/290824.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import re
from collections import namedtuple

class BasicCompiler:
    def __init__(self):
        self.variables = {}
        self.registers = ['eax', 'ebx', 'ecx', 'edx']
        self.temp_var_count = 0
        self.stack_size = 0
        self.variable_offsets = {}

    def lexical_analysis(self, source_code):
        tokens = []
        token_specification = [
            ('NUMBER',     r'\d+(\.\d*)?'),           # Integer or decimal number
            ('KEYWORD',    r'\b(if|else|while|for|do|return|int|float|char|void|break|continue|switch|case|default)\b'),  # Keywords
            ('IDENT',      r'[A-Za-z_]\w*'),          # Identifiers
            ('OP',         r'[+\-*/%]'),              # Arithmetic operators
            ('ASSIGN',     r'='),                     # Assignment operator
            ('EQ',         r'=='),                    # Equal operator
            ('NEQ',        r'!='),                    # Not equal operator
            ('LT',         r'<'),                     # Less than operator
            ('GT',         r'>'),                     # Greater than operator
            ('LTE',        r'<='),                    # Less than or equal to
            ('GTE',        r'>='),                    # Greater than or equal to
            ('AND',        r'&&'),                    # Logical AND
            ('OR',         r'\|\|'),                  # Logical OR
            ('NOT',        r'!'),                     # Logical NOT
            ('BITAND',     r'&'),                     # Bitwise AND
            ('BITOR',      r'\|'),                    # Bitwise OR
            ('BITXOR',     r'\^'),                    # Bitwise XOR
            ('BITNOT',     r'~'),                     # Bitwise NOT
            ('LSHIFT',     r'<<'),                    # Left shift
            ('RSHIFT',     r'>>'),                    # Right shift
            ('INCREMENT',  r'\+\+'),                  # Increment
            ('DECREMENT',  r'--'),                    # Decrement
            ('SEMI',       r';'),                     # Statement terminator
            ('LPAREN',     r'\('),                    # Left parenthesis
            ('RPAREN',     r'\)'),                    # Right parenthesis
            ('LBRACE',     r'\{'),                    # Left brace
            ('RBRACE',     r'\}'),                    # Right brace
            ('LBRACKET',   r'\['),                    # Left bracket
            ('RBRACKET',   r'\]'),                    # Right bracket
            ('COMMA',      r','),                     # Comma
            ('DOT',        r'\.'),                    # Dot (for member access)
            ('WHITESPACE', r'[ \t]+'),                # Skip over spaces and tabs
            ('NEWLINE',    r'\n'),                    # Line endings
            ('MISMATCH',   r'.'),                     # Any other character
        ]
        tok_regex = '|'.join('(?P<%s>%s)' % pair for pair in token_specification)
        for mo in re.finditer(tok_regex, source_code):
            kind = mo.lastgroup
            value = mo.group()
            if kind == 'NUMBER':
                value = float(value) if '.' in value else int(value)
            elif kind == 'WHITESPACE' or kind == 'NEWLINE':
                continue
            elif kind == 'MISMATCH':
                raise RuntimeError(f'{value!r} unexpected')
            tokens.append((kind, value))
        return tokens

    def syntax_analysis(self, tokens):
        ASTNode = namedtuple('ASTNode', ['type', 'value', 'children'])

        class TokenList:
            def __init__(self, tokens):
                self.tokens = tokens
                self.position = 0

            def current(self):
                return self.tokens[self.position] if self.position < len(self.tokens) else ('EOF', None)

            def next(self):
                self.position += 1
                return self.current()

            def peek(self):
                return self.tokens[self.position + 1] if self.position + 1 < len(self.tokens) else ('EOF', None)

            def __bool__(self):
                return self.position < len(self.tokens)

        tokens = TokenList(tokens)

        def parse_expression():
            def parse_primary():
                current = tokens.current()
                if current[0] == 'NUMBER':
                    tokens.next()
                    return ASTNode(type='Literal', value=current[1], children=[])
                elif current[0] == 'IDENT':
                    tokens.next()
                    return ASTNode(type='Variable', value=current[1], children=[])
                elif current[0] == 'LPAREN':
                    tokens.next()
                    expr = parse_expression()
                    if tokens.current()[0] != 'RPAREN':
                        raise SyntaxError("Expected ')'")
                    tokens.next()
                    return expr
                raise SyntaxError("Invalid expression")

            def parse_unary():
                if tokens.current()[0] in ['INCREMENT', 'DECREMENT', 'NOT', 'BITNOT']:
                    op = tokens.next()[0]
                    expr = parse_unary()
                    return ASTNode(type='UnaryOp', value=op, children=[expr])
                return parse_primary()

            def parse_binary(parse_func, ops):
                left = parse_func()
                while tokens.current()[0] in ops:
                    op = tokens.next()[0]
                    right = parse_func()
                    left = ASTNode(type='BinaryOp', value=op, children=[left, right])
                return left

            def parse_comparison():
                left = parse_binary(parse_additive, ['OP'])
                if tokens.current()[0] in ['LT', 'GT', 'LTE', 'GTE', 'EQ', 'NEQ']:
                    op = tokens.next()[0]
                    right = parse_binary(parse_additive, ['OP'])
                    return ASTNode(type='Comparison', value=op, children=[left, right])
                return left

            def parse_logical_and():
                left = parse_comparison()
                while tokens.current()[0] == 'AND':
                    op = tokens.next()[0]
                    right = parse_comparison()
                    left = ASTNode(type='LogicalOp', value=op, children=[left, right])
                return left

            def parse_logical_or():
                left = parse_logical_and()
                while tokens.current()[0] == 'OR':
                    op = tokens.next()[0]
                    right = parse_logical_and()
                    left = ASTNode(type='LogicalOp', value=op, children=[left, right])
                return left

            parse_additive = lambda: parse_binary(parse_unary, ['OP'])

            return parse_logical_or()

        def parse_statement():
            current = tokens.current()
            if current[0] == 'KEYWORD':
                if current[1] in ['int', 'float', 'char', 'void']:
                    return parse_declaration()
                elif current[1] == 'if':
                    return parse_if_statement()
                elif current[1] == 'while':
                    return parse_while_statement()
                elif current[1] == 'for':
                    return parse_for_statement()
                elif current[1] == 'return':
                    return parse_return_statement()
            elif current[0] == 'IDENT':
                return parse_assignment()
            raise SyntaxError(f"Unknown statement starting with {current}")

        def parse_declaration():
            var_type = tokens.next()[1]
            var_name = tokens.next()[1]
            if tokens.current()[0] == 'LPAREN':
                # This is a function declaration
                tokens.next()  # consume '('
                params = []
                while tokens.current()[0] != 'RPAREN':
                    if tokens.current()[0] == 'KEYWORD':
                        param_type = tokens.next()[1]
                        param_name = tokens.next()[1]
                        params.append(ASTNode(type='Parameter', value=(param_type, param_name), children=[]))
                        if tokens.current()[0] == 'COMMA':
                            tokens.next()
                    elif tokens.current()[0] != 'RPAREN':
                        raise SyntaxError("Expected parameter or ')'")
                tokens.next()  # consume ')'
                if tokens.current()[0] == 'LBRACE':
                    body = parse_block()
                    return ASTNode(type='FunctionDeclaration', value=(var_type, var_name), children=[ASTNode(type='Parameters', value=None, children=params), body])
                else:
                    raise SyntaxError("Expected '{' after function declaration")
            elif tokens.current()[0] == 'ASSIGN':
                tokens.next()
                init_expr = parse_expression()
                if tokens.current()[0] != 'SEMI':
                    raise SyntaxError("Expected ';'")
                tokens.next()
                return ASTNode(type='Declaration', value=(var_type, var_name), children=[init_expr])
            elif tokens.current()[0] == 'SEMI':
                tokens.next()
                return ASTNode(type='Declaration', value=(var_type, var_name), children=[])
            raise SyntaxError("Invalid declaration")

        def parse_assignment():
            var_name = tokens.next()[1]
            if tokens.current()[0] != 'ASSIGN':
                raise SyntaxError("Expected '='")
            tokens.next()
            expr = parse_expression()
            if tokens.current()[0] != 'SEMI':
                raise SyntaxError("Expected ';'")
            tokens.next()
            return ASTNode(type='Assignment', value='=', children=[ASTNode(type='Variable', value=var_name, children=[]), expr])

        def parse_if_statement():
            tokens.next()  # consume 'if'
            if tokens.current()[0] != 'LPAREN':
                raise SyntaxError("Expected '(' after 'if'")
            tokens.next()
            condition = parse_expression()
            if tokens.current()[0] != 'RPAREN':
                raise SyntaxError("Expected ')' after condition")
            tokens.next()
            if_body = parse_block_or_statement()
            if tokens.current()[0] == 'KEYWORD' and tokens.current()[1] == 'else':
                tokens.next()
                else_body = parse_block_or_statement()
                return ASTNode(type='IfStatement', value=None, children=[condition, if_body, else_body])
            return ASTNode(type='IfStatement', value=None, children=[condition, if_body])

        def parse_while_statement():
            tokens.next()  # consume 'while'
            if tokens.current()[0] != 'LPAREN':
                raise SyntaxError("Expected '(' after 'while'")
            tokens.next()
            condition = parse_expression()
            if tokens.current()[0] != 'RPAREN':
                raise SyntaxError("Expected ')' after condition")
            tokens.next()
            body = parse_block_or_statement()
            return ASTNode(type='WhileStatement', value=None, children=[condition, body])

        def parse_for_statement():
            tokens.next()  # consume 'for'
            if tokens.current()[0] != 'LPAREN':
                raise SyntaxError("Expected '(' after 'for'")
            tokens.next()
            init = parse_statement()
            condition = parse_expression()
            if tokens.current()[0] != 'SEMI':
                raise SyntaxError("Expected ';' after for condition")
            tokens.next()
            update = parse_expression()
            if tokens.current()[0] != 'RPAREN':
                raise SyntaxError("Expected ')' after for clauses")
            tokens.next()
            body = parse_block_or_statement()
            return ASTNode(type='ForStatement', value=None, children=[init, condition, update, body])

        def parse_return_statement():
            tokens.next()  # consume 'return'
            expr = parse_expression()
            if tokens.current()[0] != 'SEMI':
                raise SyntaxError("Expected ';' after return statement")
            tokens.next()
            return ASTNode(type='ReturnStatement', value=None, children=[expr])

        def parse_block_or_statement():
            if tokens.current()[0] == 'LBRACE':
                return parse_block()
            return parse_statement()

        def parse_block():
            if tokens.current()[0] != 'LBRACE':
                raise SyntaxError("Expected '{' at start of block")
            tokens.next()
            statements = []
            while tokens.current()[0] != 'RBRACE':
                stmt = parse_statement()
                statements.append(stmt)
            tokens.next()  # consume '}'
            return ASTNode(type='Block', value=None, children=statements)

        def parse_program():
            statements = []
            while tokens:
                stmt = parse_statement()
                statements.append(stmt)
            return ASTNode(type='Program', value=None, children=statements)

        return parse_program()

    def semantic_analysis(self, ast):
        symbol_table = {}
        loop_stack = []

        def check_node(node):
            if node.type == 'Program':
                for child in node.children:
                    check_node(child)
            elif node.type == 'Declaration':
                var_type, var_name = node.value
                if var_name in symbol_table:
                    raise NameError(f"Variable {var_name} already declared")
                symbol_table[var_name] = var_type
                if node.children:
                    init_type = check_node(node.children[0])
                    if init_type != var_type:
                        raise TypeError(f"Type mismatch in initialization of variable {var_name}")
            elif node.type == 'Assignment':
                var_name = node.children[0].value
                if var_name not in symbol_table:
                    raise NameError(f"Variable {var_name} not declared")
                var_type = symbol_table[var_name]
                expr_type = check_node(node.children[1])
                if var_type != expr_type:
                    raise TypeError(f"Type mismatch in assignment to variable {var_name}")
            elif node.type == 'BinaryOp':
                left_type = check_node(node.children[0])
                right_type = check_node(node.children[1])
                if left_type != right_type:
                    raise TypeError("Type mismatch in binary operation")
                return left_type
            elif node.type == 'UnaryOp':
                operand_type = check_node(node.children[0])
                if node.value in ['INCREMENT', 'DECREMENT']:
                    if operand_type != 'int':
                        raise TypeError(f"Cannot apply {node.value} to non-integer type")
                return operand_type
            elif node.type == 'Comparison' or node.type == 'LogicalOp':
                left_type = check_node(node.children[0])
                right_type = check_node(node.children[1])
                if left_type != right_type:
                    raise TypeError("Type mismatch in comparison or logical operation")
                return 'bool'
            elif node.type == 'Literal':
                return 'int' if isinstance(node.value, int) else 'float'
            elif node.type == 'Variable':
                if node.value not in symbol_table:
                    raise NameError(f"Variable {node.value} not declared")
                return symbol_table[node.value]
            elif node.type == 'IfStatement':
                condition_type = check_node(node.children[0])
                if condition_type != 'bool':
                    raise TypeError("Condition in if statement must be of type bool")
                check_node(node.children[1])
                if len(node.children) > 2:
                    check_node(node.children[2])
            elif node.type == 'WhileStatement':
                loop_stack.append('while')
                condition_type = check_node(node.children[0])
                if condition_type != 'bool':
                    raise TypeError("Condition in while statement must be of type bool")
                check_node(node.children[1])
                loop_stack.pop()
            elif node.type == 'ForStatement':
                loop_stack.append('for')
                check_node(node.children[0])  # initialization
                condition_type = check_node(node.children[1])
                if condition_type != 'bool':
                    raise TypeError("Condition in for statement must be of type bool")
                check_node(node.children[2])  # update
                check_node(node.children[3])  # body
                loop_stack.pop()
            elif node.type == 'ReturnStatement':
                return check_node(node.children[0])
            elif node.type == 'Block':
                for child in node.children:
                    check_node(child)

        check_node(ast)
        return ast

    def generate_ir(self, ast):
        ir = []
        temp_var_count = 0
        label_count = 0

        def get_temp_var():
            nonlocal temp_var_count
            temp_var = f"t{temp_var_count}"
            temp_var_count += 1
            return temp_var

        def get_label():
            nonlocal label_count
            label = f"L{label_count}"
            label_count += 1
            return label

        def generate_node_ir(node):
            nonlocal ir

            if node.type == 'Program':
                for child in node.children:
                    generate_node_ir(child)

            elif node.type == 'FunctionDeclaration':
                func_type, func_name = node.value
                ir.append(('FUNC_BEGIN', func_name))
                # Generate IR for parameters if needed
                for child in node.children:
                    if child.type == 'Parameters':
                        for param in child.children:
                            ir.append(('PARAM', param.value[1]))  # param.value is (type, name)
                    elif child.type == 'Block':
                        generate_node_ir(child)
                ir.append(('FUNC_END', func_name))

            elif node.type == 'Declaration':
                var_type, var_name = node.value
                ir.append(('DECLARE', var_type, var_name))
                if node.children:
                    value_ir = generate_node_ir(node.children[0])
                    ir.append(('STORE', var_name, value_ir))

            elif node.type == 'Assignment':
                var_name = node.children[0].value
                value_ir = generate_node_ir(node.children[1])
                ir.append(('STORE', var_name, value_ir))

            elif node.type == 'BinaryOp':
                left_ir = generate_node_ir(node.children[0])
                right_ir = generate_node_ir(node.children[1])
                result_var = get_temp_var()
                ir.append(('BIN_OP', node.value, left_ir, right_ir, result_var))
                return result_var

            elif node.type == 'Comparison':
                left_ir = generate_node_ir(node.children[0])
                right_ir = generate_node_ir(node.children[1])
                result_var = get_temp_var()
                ir.append(('CMP', node.value, left_ir, right_ir, result_var))
                return result_var

            elif node.type == 'Literal':
                temp_var = get_temp_var()
                ir.append(('LOAD_CONST', node.value, temp_var))
                return temp_var

            elif node.type == 'Variable':
                return node.value

            elif node.type == 'IfStatement':
                condition_ir = generate_node_ir(node.children[0])
                else_label = get_label()
                end_label = get_label()
                ir.append(('JMP_IF_FALSE', condition_ir, else_label))
                generate_node_ir(node.children[1])
                ir.append(('JMP', end_label))
                ir.append(('LABEL', else_label))
                if len(node.children) > 2:
                    generate_node_ir(node.children[2])
                ir.append(('LABEL', end_label))

            elif node.type == 'Block':
                for child in node.children:
                    generate_node_ir(child)

            elif node.type == 'ReturnStatement':
                value_ir = generate_node_ir(node.children[0])
                ir.append(('RETURN', value_ir))

            return None

        generate_node_ir(ast)
        return ir

    def optimize_ir(self, ir):
        optimized_ir = []
        constant_vars = {}
        used_vars = set()

        def is_constant(value):
            return isinstance(value, (int, float)) or (isinstance(value, str) and value in constant_vars)

        def get_constant_value(value):
            return value if isinstance(value, (int, float)) else constant_vars[value]

        def fold_constants(op, left, right):
            if op == '+':
                return left + right
            elif op == '-':
                return left - right
            elif op == '*':
                return left * right
            elif op == '/':
                return left / right
            elif op == '%':
                return left % right
            elif op == '<<':
                return left << right
            elif op == '>>':
                return left >> right
            elif op == '&':
                return left & right
            elif op == '|':
                return left | right
            elif op == '^':
                return left ^ right

        def optimize_instruction(inst):
            if inst[0] == 'LOAD_CONST':
                constant_vars[inst[2]] = inst[1]
                return inst
            elif inst[0] == 'BIN_OP':
                op, left, right, result = inst[1:]
                if is_constant(left) and is_constant(right):
                    value = fold_constants(op, get_constant_value(left), get_constant_value(right))
                    constant_vars[result] = value
                    return ('LOAD_CONST', value, result)
            elif inst[0] == 'UNARY_OP':
                op, operand, result = inst[1:]
                if is_constant(operand):
                    value = get_constant_value(operand)
                    if op == 'INCREMENT':
                        value += 1
                    elif op == 'DECREMENT':
                        value -= 1
                    elif op == 'NOT':
                        value = not value
                    elif op == 'BITNOT':
                        value = ~value
                    constant_vars[result] = value
                    return ('LOAD_CONST', value, result)
            elif inst[0] == 'CMP':
                op, left, right, result = inst[1:]
                if is_constant(left) and is_constant(right):
                    left_val, right_val = get_constant_value(left), get_constant_value(right)
                    value = False
                    if op == 'EQ':
                        value = (left_val == right_val)
                    elif op == 'NEQ':
                        value = (left_val != right_val)
                    elif op == 'LT':
                        value = (left_val < right_val)
                    elif op == 'GT':
                        value = (left_val > right_val)
                    elif op == 'LTE':
                        value = (left_val <= right_val)
                    elif op == 'GTE':
                        value = (left_val >= right_val)
                    elif op == 'AND':
                        value = (left_val and right_val)
                    elif op == 'OR':
                        value = (left_val or right_val)
                    constant_vars[result] = value
                    return ('LOAD_CONST', value, result)
            elif inst[0] == 'JMP_IF_FALSE':
                condition, label = inst[1:]
                if is_constant(condition):
                    if not get_constant_value(condition):
                        return ('JMP', label)
                    else:
                        return None  # Remove this jump entirely
            return inst

        # First pass: constant propagation and folding
        for inst in ir:
            optimized_inst = optimize_instruction(inst)
            if optimized_inst:
                optimized_ir.append(optimized_inst)
                if inst[0] in ['STORE', 'BIN_OP', 'UNARY_OP', 'CMP']:
                    used_vars.add(inst[-1])
                elif inst[0] == 'JMP_IF_FALSE':
                    used_vars.add(inst[1])

        # Second pass: dead code elimination
        final_ir = []
        for inst in optimized_ir:
            if inst[0] == 'STORE' and inst[1] not in used_vars:
                continue  # Skip storing to unused variables
            if inst[0] == 'LOAD_CONST' and inst[2] not in used_vars:
                continue  # Skip loading unused constants
            final_ir.append(inst)

        # Third pass: control flow optimization
        optimized_ir = []
        i = 0
        while i < len(final_ir):
            if i + 1 < len(final_ir) and final_ir[i][0] == 'JMP' and final_ir[i+1][0] == 'LABEL':
                if final_ir[i][1] == final_ir[i+1][1]:
                    # Skip unnecessary jump to the next instruction
                    optimized_ir.append(final_ir[i+1])
                    i += 2
                    continue

            if final_ir[i][0] == 'LABEL':
                # Check if this label is used
                label = final_ir[i][1]
                if any(inst[0] in ['JMP', 'JMP_IF_FALSE'] and inst[-1] == label for inst in final_ir):
                    optimized_ir.append(final_ir[i])
            else:
                optimized_ir.append(final_ir[i])

            i += 1

        return optimized_ir

    def code_generation(self, optimized_ir):
        assembly_code = []
        stack_size = 0
        variable_offsets = {}

        def allocate_stack(size):
            nonlocal stack_size
            stack_size += size
            return stack_size

        def get_variable_location(var):
            if var not in variable_offsets:
                variable_offsets[var] = allocate_stack(4)
            return f"[ebp-{variable_offsets[var]}]"

        for instruction in optimized_ir:
            op = instruction[0]

            if op == 'FUNC_BEGIN':
                func_name = instruction[1]
                assembly_code.append(f"{func_name}:")
                assembly_code.append("PUSH ebp")
                assembly_code.append("MOV ebp, esp")
                stack_size = 0
                variable_offsets.clear()

            elif op == 'FUNC_END':
                if stack_size > 0:
                    assembly_code.append(f"ADD esp, {stack_size}")
                assembly_code.append("POP ebp")
                assembly_code.append("RET")

            elif op == 'PARAM':
                param_name = instruction[1]
                variable_offsets[param_name] = -4 * (len(variable_offsets) + 2)  # +2 for saved ebp and return address

            elif op == 'DECLARE':
                var_type, var_name = instruction[1], instruction[2]
                get_variable_location(var_name)  # This will allocate space on the stack

            elif op == 'LOAD_CONST':
                value, dest = instruction[1], instruction[2]
                assembly_code.append(f"MOV eax, {value}")
                assembly_code.append(f"MOV {get_variable_location(dest)}, eax")

            elif op == 'STORE':
                src, dest = instruction[1], instruction[2]
                if isinstance(src, (int, float)):
                    assembly_code.append(f"MOV {get_variable_location(dest)}, {src}")
                else:
                    assembly_code.append(f"MOV eax, {get_variable_location(src)}")
                    assembly_code.append(f"MOV {get_variable_location(dest)}, eax")

            elif op == 'BIN_OP':
                op, left, right, result = instruction[1:]
                assembly_code.append(f"MOV eax, {get_variable_location(left)}")
                if op == '+':
                    assembly_code.append(f"ADD eax, {get_variable_location(right)}")
                elif op == '-':
                    assembly_code.append(f"SUB eax, {get_variable_location(right)}")
                elif op == '*':
                    assembly_code.append(f"IMUL eax, {get_variable_location(right)}")
                elif op == '/':
                    assembly_code.append("CDQ")
                    assembly_code.append(f"MOV ebx, {get_variable_location(right)}")
                    assembly_code.append("IDIV ebx")
                assembly_code.append(f"MOV {get_variable_location(result)}, eax")

            elif op == 'CMP':
                cmp_op, left, right, result = instruction[1:]
                assembly_code.append(f"MOV eax, {get_variable_location(left)}")
                assembly_code.append(f"CMP eax, {get_variable_location(right)}")
                if cmp_op == 'GT':
                    assembly_code.append("SETG al")
                elif cmp_op == 'LT':
                    assembly_code.append("SETL al")
                elif cmp_op == 'EQ':
                    assembly_code.append("SETE al")
                elif cmp_op == 'NEQ':
                    assembly_code.append("SETNE al")
                assembly_code.append("MOVZX eax, al")
                assembly_code.append(f"MOV {get_variable_location(result)}, eax")

            elif op == 'JMP':
                label = instruction[1]
                assembly_code.append(f"JMP {label}")

            elif op == 'JMP_IF_FALSE':
                condition, label = instruction[1:]
                assembly_code.append(f"CMP {get_variable_location(condition)}, 0")
                assembly_code.append(f"JE {label}")

            elif op == 'LABEL':
                label = instruction[1]
                assembly_code.append(f"{label}:")

            elif op == 'RETURN':
                value = instruction[1]
                assembly_code.append(f"MOV eax, {get_variable_location(value)}")
                if stack_size > 0:
                    assembly_code.append(f"ADD esp, {stack_size}")
                assembly_code.append("POP ebp")
                assembly_code.append("RET")

        return '\n'.join(assembly_code)


    def assemble(self, assembly_code):
        machine_code = []
        labels = {}
        unresolved_jumps = []

        def encode_register(reg):
            registers = {'eax': '000', 'ecx': '001', 'edx': '010', 'ebx': '011', 'esp': '100', 'ebp': '101', 'esi': '110', 'edi': '111'}
            return registers.get(reg.lower().rstrip(','), '000')  # Default to eax if unknown, remove trailing comma

        def encode_operand(operand):
            operand = operand.strip()
            if operand.startswith('[') and operand.endswith(']'):
                # Memory access
                return f"10{encode_register('ebp')}100"  # Assuming all memory accesses are relative to ebp
            elif operand.lower().rstrip(',') in ['eax', 'ebx', 'ecx', 'edx', 'esp', 'ebp', 'esi', 'edi']:
                # Register
                return f"01{encode_register(operand)}"
            else:
                # Immediate value or label
                try:
                    return f"11{int(operand):08b}"
                except ValueError:
                    # It's probably a label, we'll resolve it later
                    return operand

        instructions = assembly_code.split('\n')
        current_address = 0

        for instruction in instructions:
            parts = instruction.strip().split()
            if not parts:
                continue

            if parts[0].endswith(':'):
                # Label
                labels[parts[0][:-1]] = current_address
                continue

            opcode = parts[0].lower()
            operands = parts[1:]

            if opcode == 'mov':
                machine_code.append(f"0001 {encode_operand(operands[0])} {encode_operand(operands[1])}")
            elif opcode == 'add':
                machine_code.append(f"0010 {encode_operand(operands[0])} {encode_operand(operands[1])}")
            elif opcode == 'sub':
                machine_code.append(f"0011 {encode_operand(operands[0])} {encode_operand(operands[1])}")
            elif opcode == 'imul':
                machine_code.append(f"0100 {encode_operand(operands[0])} {encode_operand(operands[1])}")
            elif opcode == 'idiv':
                machine_code.append(f"0101 {encode_operand(operands[0])}")
            elif opcode == 'and':
                machine_code.append(f"0110 {encode_operand(operands[0])} {encode_operand(operands[1])}")
            elif opcode == 'or':
                machine_code.append(f"0111 {encode_operand(operands[0])} {encode_operand(operands[1])}")
            elif opcode == 'xor':
                machine_code.append(f"1000 {encode_operand(operands[0])} {encode_operand(operands[1])}")
            elif opcode == 'not':
                machine_code.append(f"1001 {encode_operand(operands[0])}")
            elif opcode == 'shl':
                machine_code.append(f"1010 {encode_operand(operands[0])} {encode_operand(operands[1])}")
            elif opcode == 'sar':
                machine_code.append(f"1011 {encode_operand(operands[0])} {encode_operand(operands[1])}")
            elif opcode == 'cmp':
                machine_code.append(f"1100 {encode_operand(operands[0])} {encode_operand(operands[1])}")
            elif opcode in ['je', 'jne', 'jl', 'jle', 'jg', 'jge']:
                unresolved_jumps.append((len(machine_code), opcode, operands[0]))
                machine_code.append(f"1101 {opcode.upper()} 00000000")  # Placeholder for jump address
            elif opcode == 'jmp':
                unresolved_jumps.append((len(machine_code), 'JMP', operands[0]))
                machine_code.append(f"1110 00000000")  # Placeholder for jump address
            elif opcode == 'call':
                unresolved_jumps.append((len(machine_code), 'CALL', operands[0]))
                machine_code.append(f"1111 00000000")  # Placeholder for call address
            elif opcode == 'ret':
                machine_code.append("10000")
            elif opcode == 'push':
                machine_code.append(f"10001 {encode_operand(operands[0])}")
            elif opcode == 'pop':
                machine_code.append(f"10010 {encode_operand(operands[0])}")
            elif opcode == 'inc':
                machine_code.append(f"10011 {encode_operand(operands[0])}")
            elif opcode == 'dec':
                machine_code.append(f"10100 {encode_operand(operands[0])}")
            elif opcode in ['sete', 'setne', 'setl', 'setle', 'setg', 'setge']:
                machine_code.append(f"10101 {opcode.upper()} {encode_operand(operands[0])}")
            elif opcode == 'movzx':
                machine_code.append(f"10110 {encode_operand(operands[0])} {encode_operand(operands[1])}")
            elif opcode == 'cdq':
                machine_code.append("10111")
            else:
                raise ValueError(f"Unknown instruction: {instruction}")

            current_address += 1

        # Resolve jumps
        for index, jump_type, label in unresolved_jumps:
            if label not in labels:
                raise ValueError(f"Undefined label: {label}")
            jump_address = labels[label]
            machine_code[index] = machine_code[index][:5] + f" {jump_address:08b}"

        return '\n'.join(machine_code)

    def compile(self, source_code):
        # Step 1: Lexical Analysis
        tokens = self.lexical_analysis(source_code)
        print("Tokens:", tokens)

        # Step 2: Syntax Analysis
        ast = self.syntax_analysis(tokens)
        print("Abstract Syntax Tree:")
        self.print_ast(ast)

        # Step 3: Semantic Analysis
        semantically_correct_ast = self.semantic_analysis(ast)
        print("Semantically checked AST:")
        self.print_ast(semantically_correct_ast)

        # Step 4: Intermediate Code Generation
        ir = self.generate_ir(semantically_correct_ast)
        print("Intermediate Representation:")
        self.print_ir(ir)

        # Step 5: Optimization
        optimized_ir = self.optimize_ir(ir)
        print("Optimized Intermediate Representation:")
        self.print_ir(optimized_ir)

        # Step 6: Code Generation
        assembly_code = self.code_generation(optimized_ir)
        print("Assembly Code:")
        print(assembly_code)

        # Step 7: Assembly
        machine_code = self.assemble(assembly_code)
        print("Machine Code:")
        print(machine_code)

        return machine_code

    def print_ast(self, node, indent=""):
        print(f"{indent}{node.type}: {node.value}")
        for child in node.children:
            self.print_ast(child, indent + "  ")

    def print_ir(self, ir):
        for instruction in ir:
            print(instruction)


# Test the compiler with a more complex C-like code
compiler = BasicCompiler()
source_code = """
int main() {
    int x = 5;
    int y = 10;
    int z = x + y;
    if (z > 10) {
        return z;
    } else {
        return x;
    }
}
"""
machine_code = compiler.compile(source_code)


Tokens: [('KEYWORD', 'int'), ('IDENT', 'main'), ('LPAREN', '('), ('RPAREN', ')'), ('LBRACE', '{'), ('KEYWORD', 'int'), ('IDENT', 'x'), ('ASSIGN', '='), ('NUMBER', 5), ('SEMI', ';'), ('KEYWORD', 'int'), ('IDENT', 'y'), ('ASSIGN', '='), ('NUMBER', 10), ('SEMI', ';'), ('KEYWORD', 'int'), ('IDENT', 'z'), ('ASSIGN', '='), ('IDENT', 'x'), ('OP', '+'), ('IDENT', 'y'), ('SEMI', ';'), ('KEYWORD', 'if'), ('LPAREN', '('), ('IDENT', 'z'), ('GT', '>'), ('NUMBER', 10), ('RPAREN', ')'), ('LBRACE', '{'), ('KEYWORD', 'return'), ('IDENT', 'z'), ('SEMI', ';'), ('RBRACE', '}'), ('KEYWORD', 'else'), ('LBRACE', '{'), ('KEYWORD', 'return'), ('IDENT', 'x'), ('SEMI', ';'), ('RBRACE', '}'), ('RBRACE', '}')]
Abstract Syntax Tree:
Program: None
  FunctionDeclaration: ('main', '(')
    Parameters: None
    Block: None
      Declaration: ('x', '=')
        Literal: 5
      Declaration: ('y', '=')
        Literal: 10
      Declaration: ('z', '=')
        BinaryOp: IDENT
          Variable: x
          Variable: y
  